In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, GRU, RNN, Embedding, Dense, Attention, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the model parameters
vocab_size = 10000  # Vocabulary size
embedding_dim = 256  # Embedding dimension
max_seq_len = 30  # Max sequence length
latent_dim = 512  # Latent dimensionality of the encoding space

# Encoder (RNN, LSTM, or GRU)
def build_encoder(input_dim, latent_dim, seq_len):
    encoder_inputs = Input(shape=(seq_len,))
    x = Embedding(input_dim=input_dim, output_dim=embedding_dim)(encoder_inputs)
    x = LSTM(latent_dim, return_state=True)(x)
    encoder_outputs, state_h, state_c = x
    return encoder_inputs, encoder_outputs, state_h, state_c

# Decoder with Attention mechanism
def build_decoder(input_dim, latent_dim, seq_len, encoder_outputs, state_h, state_c):
    decoder_inputs = Input(shape=(seq_len,))
    x = Embedding(input_dim=input_dim, output_dim=embedding_dim)(decoder_inputs)

    # Attention mechanism
    attention = Attention(use_scale=True)([x, encoder_outputs])
    decoder_concat = Concatenate(axis=-1)([x, attention])

    # Decoder LSTM
    x = LSTM(latent_dim, return_sequences=True)(decoder_concat, initial_state=[state_h, state_c])
    decoder_outputs = Dense(input_dim, activation='softmax')(x)

    return decoder_inputs, decoder_outputs

# Define the Seq2Seq model using LSTM (you can replace it with GRU or RNN)
def build_seq2seq_model(input_dim, latent_dim, seq_len):
    encoder_inputs, encoder_outputs, state_h, state_c = build_encoder(input_dim, latent_dim, seq_len)
    decoder_inputs, decoder_outputs = build_decoder(input_dim, latent_dim, seq_len, encoder_outputs, state_h, state_c)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the Seq2Seq model
model = build_seq2seq_model(vocab_size, latent_dim, max_seq_len)

# Print the model summary
model.summary()

# To train the model, you would need to prepare the training data (encoder_input_data, decoder_input_data, decoder_target_data)
# For example:
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data, epochs=10, batch_size=64)



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 30, 256)   │  2,560,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 30, 256)   │  2,560,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 512),     │  1,574,912 │ embedding[0][0]   │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 30, 512)   │          1 │ embedding_1[0][0… │
│ (Attention)         │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 30, 768)   │          0 │ embedding_1[0][0… │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 30, 512)   │  2,623,488 │ concatenate[0][0… │
│                     │                   │            │ lstm[0][1],       │
│                     │                   │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 30, 10000) │  5,130,000 │ lstm_1[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,448,401 (55.12 MB)

 Trainable params: 14,448,401 (55.12 MB)

 Non-trainable params: 0 (0.00 B)